<a href="https://colab.research.google.com/github/AriyantGit/Book_recomendation/blob/main/Books_recomendation_pearson_corelation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np

In [10]:
df_books=pd.read_csv("/content/gdrive/MyDrive/Books_Data/BX-Books.csv",sep=";", encoding='latin-1',on_bad_lines='skip')

<ipython-input-10-5908358926c7>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books=pd.read_csv("/content/gdrive/MyDrive/Books_Data/BX-Books.csv",sep=";", encoding='latin-1',on_bad_lines='skip')


In [11]:
df_books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [12]:
df_user=pd.read_csv("/content/gdrive/MyDrive/Books_Data/BX-Users.csv",sep=";", encoding='latin-1',on_bad_lines='skip')

In [13]:
df_user.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [14]:
df_rating=pd.read_csv("/content/gdrive/MyDrive/Books_Data/BX-Book-Ratings.csv",sep=";", encoding='latin-1',on_bad_lines='skip')

In [16]:
df_rating.sort_values("User-ID").head()

,User-ID,ISBN,Book-Rating
9561,2,0195153448,0
9562,7,034542252,0
9572,8,0771025661,0
9580,8,1881320189,7
9579,8,1575663937,6


# **Data Preparation (Data Cleaning and Feature Engineering)**

# droping the image url from df_books data

In [22]:
df_books.drop(["Image-URL-S","Image-URL-M","Image-URL-L"],axis=1,inplace=True)
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
dtypes: object(5)
memory usage: 10.4+ MB


In [24]:
df_books.shape

(271360, 5)

### replacing '-' with '_' and features name in lower case

In [31]:
df_books.columns=df_books.columns.str.strip().str.lower().str.replace("-","_")
df_user.columns=df_user.columns.str.strip().str.lower().str.replace("-","_")
df_rating.columns=df_rating.columns.str.strip().str.lower().str.replace("-","_")